# Main

In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import cv2
import numpy as np
from PIL import Image, ImageTk
import threading
import os

try:
    from ultralytics import YOLO
    YOLO_AVAILABLE = True
except ImportError:
    YOLO_AVAILABLE = False

# ─────────────────────────────────────────────
# HSV Color Ranges for Car Color Detection
# ─────────────────────────────────────────────
COLOR_RANGES = {
    "แดง (Red)":    [((0,   60, 60),  (10,  255, 255)),
                     ((160, 60, 60),  (180, 255, 255))],
    "ส้ม (Orange)": [((11,  80, 80),  (25,  255, 255))],
    "เหลือง (Yellow)":[((26, 80, 80), (34,  255, 255))],
    "เขียว (Green)": [((35, 50, 50),  (85,  255, 255))],
    "ฟ้า (Cyan)":   [((86, 50, 50),  (100, 255, 255))],
    "น้ำเงิน (Blue)":[((101,50, 50),  (130, 255, 255))],
    "ม่วง (Purple)":[((131,50, 50),  (159, 255, 255))],
    "ขาว (White)":  [((0,  0,  180), (180, 30,  255))],
    "เทา (Gray)":   [((0,  0,  80),  (180, 30,  179))],
    "ดำ (Black)":   [((0,  0,  0),   (180, 255, 79))],
    "น้ำตาล (Brown)":[((8, 60, 40),  (20,  200, 160))],
    "เงิน (Silver)":[((0,  0,  160), (180, 20,  255))],
}

def detect_car_color(roi_bgr):
    """Detect dominant car color from a BGR ROI using HSV."""
    if roi_bgr is None or roi_bgr.size == 0:
        return "ไม่สามารถระบุ", {}

    hsv = cv2.cvtColor(roi_bgr, cv2.COLOR_BGR2HSV)
    total_pixels = hsv.shape[0] * hsv.shape[1]
    if total_pixels == 0:
        return "ไม่สามารถระบุ", {}

    scores = {}
    for color_name, ranges in COLOR_RANGES.items():
        mask = np.zeros(hsv.shape[:2], dtype=np.uint8)
        for (lo, hi) in ranges:
            mask |= cv2.inRange(hsv, np.array(lo), np.array(hi))
        scores[color_name] = cv2.countNonZero(mask) / total_pixels * 100

    best = max(scores, key=scores.get)
    return best, scores


def rotate_rect_points(cx, cy, w, h, angle_deg):
    """Return 4 corner points of a rotated rectangle."""
    angle = np.radians(angle_deg)
    cos_a, sin_a = np.cos(angle), np.sin(angle)
    dx, dy = w / 2, h / 2
    corners = [(-dx, -dy), (dx, -dy), (dx, dy), (-dx, dy)]
    pts = []
    for x, y in corners:
        rx = cx + x * cos_a - y * sin_a
        ry = cy + x * sin_a + y * cos_a
        pts.append([rx, ry])
    return np.array(pts, dtype=np.float32)


# ─────────────────────────────────────────────
# Main GUI Application
# ─────────────────────────────────────────────
class CarDetectionApp(tk.Tk):
    ORANGE_BG    = "#1A0A00"
    ORANGE_PANEL = "#2D1200"
    ORANGE_CARD  = "#3D1A00"
    ORANGE_MID   = "#FF6B00"
    ORANGE_LIGHT = "#FF9A3C"
    ORANGE_BRIGHT= "#FFB74D"
    ORANGE_TEXT  = "#FFF3E0"
    ORANGE_SUB   = "#FFCC80"
    ORANGE_ACCENT= "#FF4500"
    ORANGE_GLOW  = "#FF8C00"

    def __init__(self):
        super().__init__()
        self.title("🚗 Car & Logo Detection — HSV Color Analyzer")
        self.geometry("1280x820")
        self.minsize(1000, 700)
        self.configure(bg=self.ORANGE_BG)

        self.model = None
        self.current_image_path = None
        self.result_image = None
        self.detections = []

        self._load_fonts()
        self._build_styles()
        self._build_ui()
        self._try_load_model()

    # ── Fonts & Styles ──────────────────────────────────
    def _load_fonts(self):
        import tkinter.font as tkfont
        self.font_title  = tkfont.Font(family="Kanit",     size=18, weight="bold")
        self.font_header = tkfont.Font(family="Kanit",     size=12, weight="bold")
        self.font_body   = tkfont.Font(family="TH Sarabun New", size=12)
        self.font_mono   = tkfont.Font(family="Courier New", size=10)
        self.font_btn    = tkfont.Font(family="Kanit",     size=11, weight="bold")

    def _build_styles(self):
        s = ttk.Style(self)
        s.theme_use("clam")

        s.configure("Orange.TFrame",        background=self.ORANGE_PANEL)
        s.configure("Card.TFrame",          background=self.ORANGE_CARD)
        s.configure("TLabel",               background=self.ORANGE_PANEL, foreground=self.ORANGE_TEXT)
        s.configure("Header.TLabel",        background=self.ORANGE_BG,    foreground=self.ORANGE_MID,
                    font=("Kanit", 13, "bold"))
        s.configure("Sub.TLabel",           background=self.ORANGE_CARD,  foreground=self.ORANGE_SUB,
                    font=("TH Sarabun New", 11))
        s.configure("Accent.TLabel",        background=self.ORANGE_CARD,  foreground=self.ORANGE_BRIGHT,
                    font=("Kanit", 12, "bold"))
        s.configure("Status.TLabel",        background=self.ORANGE_BG,    foreground=self.ORANGE_LIGHT,
                    font=("TH Sarabun New", 10))
        s.configure("Orange.TButton",
                    background=self.ORANGE_MID, foreground="#FFFFFF",
                    font=("Kanit", 11, "bold"), padding=(12, 6),
                    borderwidth=0, relief="flat")
        s.map("Orange.TButton",
              background=[("active", self.ORANGE_GLOW), ("pressed", self.ORANGE_ACCENT)],
              foreground=[("active", "#FFFFFF")])
        s.configure("Detect.TButton",
                    background=self.ORANGE_ACCENT, foreground="#FFFFFF",
                    font=("Kanit", 12, "bold"), padding=(16, 8),
                    borderwidth=0, relief="flat")
        s.map("Detect.TButton",
              background=[("active", "#FF6020"), ("disabled", "#5A2A00")],
              foreground=[("disabled", "#996633")])

        s.configure("Orange.Treeview",
                    background=self.ORANGE_CARD,
                    foreground=self.ORANGE_TEXT,
                    fieldbackground=self.ORANGE_CARD,
                    font=("TH Sarabun New", 11),
                    rowheight=28)
        s.configure("Orange.Treeview.Heading",
                    background=self.ORANGE_MID,
                    foreground="#FFFFFF",
                    font=("Kanit", 10, "bold"),
                    relief="flat")
        s.map("Orange.Treeview",
              background=[("selected", self.ORANGE_MID)],
              foreground=[("selected", "#FFFFFF")])

        s.configure("Orange.TScrollbar",
                    background=self.ORANGE_CARD,
                    troughcolor=self.ORANGE_BG,
                    arrowcolor=self.ORANGE_MID)

    # ── UI Layout ───────────────────────────────────────
    def _build_ui(self):
        # ── Header bar ──
        header = tk.Frame(self, bg=self.ORANGE_BG, height=60)
        header.pack(fill="x", padx=0, pady=0)
        header.pack_propagate(False)

        tk.Label(header, text="🍊 CAR & LOGO DETECTOR",
                 bg=self.ORANGE_BG, fg=self.ORANGE_MID,
                 font=("Kanit", 20, "bold")).pack(side="left", padx=20, pady=8)
        tk.Label(header, text="HSV Color Analysis • YOLOv8 OBB",
                 bg=self.ORANGE_BG, fg=self.ORANGE_SUB,
                 font=("TH Sarabun New", 11)).pack(side="left", padx=4, pady=8)

        self.model_label = tk.Label(header, text="⏳ กำลังโหลดโมเดล...",
                                    bg=self.ORANGE_BG, fg=self.ORANGE_LIGHT,
                                    font=("Kanit", 10))
        self.model_label.pack(side="right", padx=20)

        # Separator line
        sep = tk.Frame(self, bg=self.ORANGE_MID, height=2)
        sep.pack(fill="x")

        # ── Main body ──
        body = tk.Frame(self, bg=self.ORANGE_BG)
        body.pack(fill="both", expand=True, padx=8, pady=8)
        body.columnconfigure(0, weight=3)
        body.columnconfigure(1, weight=2)
        body.rowconfigure(0, weight=1)

        # Left panel: image display
        self._build_image_panel(body)

        # Right panel: controls + results
        self._build_right_panel(body)

        # ── Status bar ──
        self.status_var = tk.StringVar(value="พร้อมใช้งาน — เลือกภาพเพื่อเริ่มต้น")
        statusbar = tk.Frame(self, bg=self.ORANGE_PANEL, height=28)
        statusbar.pack(fill="x", side="bottom")
        ttk.Label(statusbar, textvariable=self.status_var,
                  style="Status.TLabel").pack(side="left", padx=12, pady=4)

    def _build_image_panel(self, parent):
        frame = tk.Frame(parent, bg=self.ORANGE_PANEL, bd=0)
        frame.grid(row=0, column=0, sticky="nsew", padx=(0, 4))
        frame.rowconfigure(1, weight=1)
        frame.columnconfigure(0, weight=1)

        # Top toolbar
        toolbar = tk.Frame(frame, bg=self.ORANGE_CARD, height=48)
        toolbar.grid(row=0, column=0, sticky="ew", pady=(0, 2))
        toolbar.grid_propagate(False)

        ttk.Button(toolbar, text="📂  เลือกภาพ", style="Orange.TButton",
                   command=self._browse_image).pack(side="left", padx=8, pady=6)
        self.detect_btn = ttk.Button(toolbar, text="🔍  ตรวจจับ", style="Detect.TButton",
                                     command=self._run_detection, state="disabled")
        self.detect_btn.pack(side="left", padx=4, pady=6)
        ttk.Button(toolbar, text="💾  บันทึกผล", style="Orange.TButton",
                   command=self._save_result).pack(side="left", padx=4, pady=6)
        ttk.Button(toolbar, text="🔄  ล้างหน้าจอ", style="Orange.TButton",
                   command=self._clear).pack(side="right", padx=8, pady=6)

        # Canvas
        canvas_frame = tk.Frame(frame, bg="#0D0500", bd=2, relief="sunken")
        canvas_frame.grid(row=1, column=0, sticky="nsew")
        canvas_frame.rowconfigure(0, weight=1)
        canvas_frame.columnconfigure(0, weight=1)

        self.canvas = tk.Canvas(canvas_frame, bg="#0D0500", cursor="crosshair",
                                highlightthickness=0)
        self.canvas.grid(row=0, column=0, sticky="nsew")

        # Placeholder text
        self.canvas_hint = self.canvas.create_text(
            400, 300, text="🖼  ลากภาพมาวาง หรือคลิก 'เลือกภาพ'",
            fill=self.ORANGE_CARD, font=("Kanit", 16, "bold"), anchor="center")

        self.canvas.bind("<Configure>", self._on_canvas_resize)

    def _build_right_panel(self, parent):
        frame = tk.Frame(parent, bg=self.ORANGE_BG)
        frame.grid(row=0, column=1, sticky="nsew")
        frame.rowconfigure(1, weight=1)
        frame.columnconfigure(0, weight=1)

        # File info card
        info_card = tk.Frame(frame, bg=self.ORANGE_CARD, bd=0)
        info_card.grid(row=0, column=0, sticky="ew", pady=(0, 4))

        tk.Label(info_card, text="📌 ข้อมูลภาพ",
                 bg=self.ORANGE_CARD, fg=self.ORANGE_MID,
                 font=("Kanit", 11, "bold")).pack(anchor="w", padx=10, pady=(8, 2))

        self.file_label = tk.Label(info_card, text="—  ยังไม่ได้เลือกภาพ",
                                   bg=self.ORANGE_CARD, fg=self.ORANGE_SUB,
                                   font=("TH Sarabun New", 11), wraplength=340, justify="left")
        self.file_label.pack(anchor="w", padx=14, pady=(0, 8))

        # Results area
        result_outer = tk.Frame(frame, bg=self.ORANGE_PANEL)
        result_outer.grid(row=1, column=0, sticky="nsew")
        result_outer.rowconfigure(1, weight=1)
        result_outer.columnconfigure(0, weight=1)

        tk.Label(result_outer, text="🔎 ผลการตรวจจับ",
                 bg=self.ORANGE_PANEL, fg=self.ORANGE_MID,
                 font=("Kanit", 12, "bold")).grid(row=0, column=0, sticky="w", padx=8, pady=6)

        # Treeview
        cols = ("id", "class", "conf", "color", "pct")
        self.tree = ttk.Treeview(result_outer, columns=cols, show="headings",
                                  style="Orange.Treeview", selectmode="browse")
        self.tree.heading("id",    text="#")
        self.tree.heading("class", text="คลาส")
        self.tree.heading("conf",  text="Conf")
        self.tree.heading("color", text="สีรถ")
        self.tree.heading("pct",   text="%พื้นที่")
        self.tree.column("id",    width=30, anchor="center")
        self.tree.column("class", width=90, anchor="center")
        self.tree.column("conf",  width=60, anchor="center")
        self.tree.column("color", width=130, anchor="w")
        self.tree.column("pct",   width=70, anchor="center")
        self.tree.grid(row=1, column=0, sticky="nsew", padx=(4, 0), pady=(0, 4))

        scrolly = ttk.Scrollbar(result_outer, orient="vertical",
                                 command=self.tree.yview, style="Orange.TScrollbar")
        scrolly.grid(row=1, column=1, sticky="ns", pady=(0, 4))
        self.tree.configure(yscrollcommand=scrolly.set)
        self.tree.bind("<<TreeviewSelect>>", self._on_detection_select)

        # Detail card
        detail_frame = tk.Frame(result_outer, bg=self.ORANGE_CARD, bd=0)
        detail_frame.grid(row=2, column=0, columnspan=2, sticky="ew", padx=4, pady=(2, 4))

        tk.Label(detail_frame, text="🎨 รายละเอียดสี (HSV)",
                 bg=self.ORANGE_CARD, fg=self.ORANGE_MID,
                 font=("Kanit", 10, "bold")).pack(anchor="w", padx=10, pady=(6, 2))

        self.detail_text = tk.Text(detail_frame, height=8, width=38,
                                   bg="#200D00", fg=self.ORANGE_LIGHT,
                                   font=("Courier New", 9),
                                   relief="flat", bd=0,
                                   state="disabled", wrap="word")
        self.detail_text.pack(fill="x", padx=8, pady=(0, 8))

    # ── Model Loading ────────────────────────────────────
    def _try_load_model(self):
        def _load():
            model_path = "best.pt"
            if not os.path.exists(model_path):
                self.after(0, lambda: self.model_label.config(
                    text="⚠️ ไม่พบ best.pt", fg="#FF4444"))
                return
            if not YOLO_AVAILABLE:
                self.after(0, lambda: self.model_label.config(
                    text="⚠️ ไม่พบ ultralytics", fg="#FF4444"))
                return
            try:
                self.model = YOLO(model_path)
                self.after(0, lambda: self.model_label.config(
                    text="✅ โมเดลพร้อม: best.pt", fg="#66FF99"))
            except Exception as e:
                self.after(0, lambda: self.model_label.config(
                    text=f"❌ โหลดโมเดลล้มเหลว: {e}", fg="#FF4444"))
        threading.Thread(target=_load, daemon=True).start()

    # ── Image Browse ─────────────────────────────────────
    def _browse_image(self):
        path = filedialog.askopenfilename(
            title="เลือกภาพ",
            filetypes=[("Image Files", "*.jpg *.jpeg *.png *.bmp *.webp *.tiff"),
                       ("All Files", "*.*")])
        if path:
            self._load_image(path)

    def _load_image(self, path):
        self.current_image_path = path
        self.detections = []
        self.result_image = None
        self._clear_tree()
        self._clear_detail()

        img = Image.open(path)
        w, h = img.size
        fname = os.path.basename(path)
        self.file_label.config(text=f"📄 {fname}\n📐 {w} × {h} px")
        self._display_pil(img)
        self.detect_btn.config(state="normal")
        self.status_var.set(f"โหลดภาพ: {fname}")

    def _display_pil(self, pil_img):
        self.canvas.delete("all")
        cw = self.canvas.winfo_width()  or 640
        ch = self.canvas.winfo_height() or 480
        if cw < 10: cw = 640
        if ch < 10: ch = 480

        pil_img.thumbnail((cw, ch), Image.LANCZOS)
        self._tk_img = ImageTk.PhotoImage(pil_img)
        self.canvas.create_image(cw//2, ch//2, anchor="center", image=self._tk_img)

    # ── Detection ────────────────────────────────────────
    def _run_detection(self):
        if not self.current_image_path:
            return
        if self.model is None:
            messagebox.showwarning("โมเดล", "ยังไม่ได้โหลดโมเดล กรุณารอสักครู่")
            return

        self.detect_btn.config(state="disabled")
        self.status_var.set("🔄 กำลังตรวจจับ...")
        threading.Thread(target=self._detect_thread, daemon=True).start()

    def _detect_thread(self):
        try:
            img_bgr = cv2.imread(self.current_image_path)
            results = self.model(self.current_image_path, verbose=False)
            detections = []

            for result in results:
                # Try OBB first, fallback to boxes
                if hasattr(result, 'obb') and result.obb is not None:
                    boxes_data = result.obb
                    is_obb = True
                else:
                    boxes_data = result.boxes
                    is_obb = False

                names = result.names
                if boxes_data is None:
                    continue

                for i, box in enumerate(boxes_data):
                    conf = float(box.conf[0])
                    cls_id = int(box.cls[0])
                    cls_name = names.get(cls_id, str(cls_id))

                    # Extract ROI
                    if is_obb:
                        try:
                            # xywhr format: cx, cy, w, h, angle
                            xywhr = box.xywhr[0].cpu().numpy()
                            cx, cy, w, h, angle = xywhr
                            pts = rotate_rect_points(cx, cy, w, h, np.degrees(angle))
                            roi = self._get_obb_roi(img_bgr, pts)
                        except Exception:
                            xyxy = box.xyxy[0].cpu().numpy().astype(int)
                            roi = img_bgr[xyxy[1]:xyxy[3], xyxy[0]:xyxy[2]]
                            pts = None
                    else:
                        xyxy = box.xyxy[0].cpu().numpy().astype(int)
                        roi = img_bgr[xyxy[1]:xyxy[3], xyxy[0]:xyxy[2]]
                        pts = xyxy

                    color_name, color_scores = detect_car_color(roi)
                    roi_size = roi.shape[0] * roi.shape[1] if roi.size > 0 else 0
                    total_size = img_bgr.shape[0] * img_bgr.shape[1]
                    pct = roi_size / total_size * 100 if total_size > 0 else 0

                    detections.append({
                        "id": i + 1,
                        "class": cls_name,
                        "conf": conf,
                        "color": color_name,
                        "pct": pct,
                        "color_scores": color_scores,
                        "pts": pts,
                        "is_obb": is_obb,
                    })

            # Draw result image
            result_bgr = img_bgr.copy()
            for det in detections:
                self._draw_detection(result_bgr, det)

            result_rgb = cv2.cvtColor(result_bgr, cv2.COLOR_BGR2RGB)
            self.result_image = Image.fromarray(result_rgb)
            self.detections = detections

            self.after(0, self._update_ui_after_detect)
        except Exception as e:
            self.after(0, lambda: (
                self.status_var.set(f"❌ เกิดข้อผิดพลาด: {e}"),
                self.detect_btn.config(state="normal"),
                messagebox.showerror("Error", str(e))
            ))

    def _get_obb_roi(self, img, pts):
        """Warp OBB region to a straight rectangle for color analysis."""
        pts = pts.astype(np.float32)
        rect = cv2.boundingRect(pts)
        x, y, w, h = rect
        x, y = max(0, x), max(0, y)
        w = min(w, img.shape[1] - x)
        h = min(h, img.shape[0] - y)
        return img[y:y+h, x:x+w]

    def _draw_detection(self, img, det):
        pts = det["pts"]
        cls  = det["class"]
        conf = det["conf"]
        color_name = det["color"]

        label = f"{cls} {conf:.2f} | {color_name}"
        color_bgr = (0, 120, 255)  # orange in BGR

        if det["is_obb"] and pts is not None and len(pts) == 4:
            pts_int = pts.astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(img, [pts_int], isClosed=True, color=color_bgr, thickness=2)
            x0, y0 = int(pts[0][0]), int(pts[0][1])
        else:
            if pts is not None and len(pts) == 4:
                x0, y0, x1, y1 = pts
                cv2.rectangle(img, (x0, y0), (x1, y1), color_bgr, 2)
            else:
                x0, y0 = 10, 10

        # Label background
        (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.55, 1)
        cv2.rectangle(img, (x0, y0 - th - 8), (x0 + tw + 6, y0), color_bgr, -1)
        cv2.putText(img, label, (x0 + 3, y0 - 4),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255, 255, 255), 1, cv2.LINE_AA)

    def _update_ui_after_detect(self):
        self._clear_tree()
        for det in self.detections:
            self.tree.insert("", "end", iid=str(det["id"]), values=(
                det["id"], det["class"],
                f"{det['conf']:.2f}",
                det["color"],
                f"{det['pct']:.1f}%"
            ))
        if self.result_image:
            self._display_pil(self.result_image.copy())
        n = len(self.detections)
        self.status_var.set(f"✅ ตรวจจับเสร็จสิ้น — พบ {n} วัตถุ")
        self.detect_btn.config(state="normal")

    # ── Tree Selection ───────────────────────────────────
    def _on_detection_select(self, event):
        sel = self.tree.selection()
        if not sel:
            return
        iid = int(sel[0])
        det = next((d for d in self.detections if d["id"] == iid), None)
        if not det:
            return
        scores = det.get("color_scores", {})
        lines = [f"{'สี':<20} {'%พื้นที่':>8}", "─" * 30]
        for name, pct in sorted(scores.items(), key=lambda x: -x[1]):
            bar = "█" * int(pct / 5)
            lines.append(f"{name:<18} {pct:>6.1f}%  {bar}")
        text = "\n".join(lines)
        self.detail_text.config(state="normal")
        self.detail_text.delete("1.0", "end")
        self.detail_text.insert("end", text)
        self.detail_text.config(state="disabled")

    # ── Save ─────────────────────────────────────────────
    def _save_result(self):
        if self.result_image is None:
            messagebox.showinfo("บันทึก", "ยังไม่มีผลการตรวจจับให้บันทึก")
            return
        path = filedialog.asksaveasfilename(
            defaultextension=".jpg",
            filetypes=[("JPEG", "*.jpg"), ("PNG", "*.png")])
        if path:
            self.result_image.save(path)
            self.status_var.set(f"💾 บันทึกแล้ว: {os.path.basename(path)}")

    # ── Helpers ──────────────────────────────────────────
    def _clear(self):
        self.canvas.delete("all")
        self.canvas_hint = self.canvas.create_text(
            400, 300, text="🖼  ลากภาพมาวาง หรือคลิก 'เลือกภาพ'",
            fill=self.ORANGE_CARD, font=("Kanit", 16, "bold"), anchor="center")
        self.current_image_path = None
        self.result_image = None
        self.detections = []
        self.file_label.config(text="—  ยังไม่ได้เลือกภาพ")
        self.detect_btn.config(state="disabled")
        self._clear_tree()
        self._clear_detail()
        self.status_var.set("ล้างหน้าจอแล้ว")

    def _clear_tree(self):
        for item in self.tree.get_children():
            self.tree.delete(item)

    def _clear_detail(self):
        self.detail_text.config(state="normal")
        self.detail_text.delete("1.0", "end")
        self.detail_text.config(state="disabled")

    def _on_canvas_resize(self, event):
        if self.result_image:
            self._display_pil(self.result_image.copy())
        elif self.current_image_path:
            img = Image.open(self.current_image_path)
            self._display_pil(img)


# ─────────────────────────────────────────────
if __name__ == "__main__":
    app = CarDetectionApp()
    app.mainloop()